# Min-Code: How little code will suffice to write min-cut from scratch?

## 2024-08-26: Intro

<a href="https://adventofcode.com/2023/day/25">Day 25</a> of <a href="https://adventofcode.com/2023">Advent of Code 2023</a> describes a highly-interconnected network that can be split in two by cutting three specific wires. Our main job is to identify those three wires. This is known as finding a <a href="https://en.wikipedia.org/wiki/Minimum_cut">minimum cut</a>.

When a simple algorithm is needed, I often like to write it from scratch each time, using only built-ins and standard libraries. (Inspirations include the <a href="https://www.reddit.com/r/adventofcode/">Advent of Code subreddit</a>, <a href="https://github.com/norvig/pytudes#pytudes-index-of-jupyter-ipython-notebooks">Peter Norvig</a>, and <a href="https://www.youtube.com/watch?v=j6VSAsKAj98">David Beazley</a>.) By the sixth or seventh time I wrote a breadth-first search, I did it without any conscious doubt about how I was doing it. But that was BFS.

I read about maxflows and mincuts in Sedgewick's <a href="https://algs4.cs.princeton.edu/">Algorithms</a>. Their Java code for maxflow problems includes several classes but is quite general. Some classes contain many attributes and methods that I wouldn't need to solve Day 25. I wondered whether I could simplify Sedgewick's approach enough to rewrite it from scratch, in Python, and still get a decent night's sleep. My main problem was that I didn't really understand the <a href="https://en.wikipedia.org/wiki/Ford%E2%80%93Fulkerson_algorithm">Ford–Fulkerson algorithm</a>, and how to identify the residual network in Sedgewick's implementation so as to find a minimum cut. As of starting this project, I still don't.

Instead, to collect the last of my 50 stars that night — disclaimer: it was January 30, not December 24 — I took a brute-force approach and eventually came up with a simplifying assumption that, although not guaranteed to produce the correct answer, did so in under ten seconds runtime. A few months later, I used <a href="https://networkx.org/documentation/stable/">NetworkX</a> to write an almost trivial solution to the problem, letting the `minimum_edge_cut` and `connected_components` functions do the hard work. NetworkX seems pretty great, and there's probably no reason to avoid using it if I'm already working in Python.

But now it's time to return to my original objective. Can I simplify min-cut enough so that I can write it from scratch as needed?

## 2024-08-27: Initial solution

Before diving any deeper, I'll describe a cleaned-up version of my <a href="https://github.com/RobinFiveWords/AdventOfCode/blob/main/2023/day25.py">initial solution</a>. If we accept that our starting graph (network) has just one connected component — from any given vertex we can reach every other vertex — then we can test any potential solution by removing its three edges (wires) from the graph and rechecking whether, in this modified graph, from any given vertex we can reach every other vertex. To check the modified graph, we can arbitrarily pick a vertex and do a depth-first search, counting how many vertices we reach. If it is less than the total number of vertices, we have our solution.

I couldn't just brute-force it. My puzzle input had 1,514 vertices connected by 3,385 edges, meaning I would potentially have to test
$$
\frac{3{,}385 \times 3{,}384 \times 3{,}383}{3 \times 2 \times 1} = 38{,}751{,}723{,}720 \nonumber
$$
combinations of three edges. So I looked for a way to reduce the number of possible solutions I would consider.

The assumption I came up with was that the three edges in the solution would connect vertices — let's call these the solution vertices — that were "closer" to all other vertices, on average, than most other vertices were. In other words, if for any given vertex I added up the shortest number of edges that would need to be traversed to reach each other vertex, the solution vertices would be among the vertices with the lowest sums.

For my reasoning, let's consider one of the edges in the solution to be $X{\textendash}Y$, connecting vertices $X$ and $Y$. In the modified graph, let's say $X$ is in connected component $A$, and $Y$ is in connected component $B$. We still want to focus on the original, connected graph, but with each vertex belonging to either $A$ or $B$. Now let's consider all the vertices in $A$ other than $X$ for which $X{\textendash}Y$ is the closest solution edge. For these vertices in $A$, their shortest path to reach any vertex in $B$ is through $X$, which means they are farther from every vertex in $B$ than $X$ is. If $X$ is closer to many other vertices than many others are, perhaps on average it is closer to _all_ other vertices than _most_ others are.

For each vertex, I did a breadth-first search to compute the sum of its distances from each other vertex. I then identified any edge in the graph whose vertices were both among the 100 with the lowest total distance. There were 105 such edges, meaning I would only have to test
$$
\frac{105 \times 104 \times 103}{3 \times 2 \times 1} = 187{,}460 \nonumber
$$
combinations of three edges. One of these turned out to be the solution.

## 2024-08-28: Test graph

I'm not going to understand this unless I draw an example and carry out the algorithm by hand. I decided to try using the Day 25 test example, which has 15 vertices and 33 edges. This may turn out to be too large to learn on, but I'm hoping that its complexity will help me test my conclusions.

I started by drawing the test graph by hand, to see whether I would have any layout issues.

<div align="center">
    <img src="img/20240828_1.jpeg"
        style="max-height:400px"
        alt="Hand-drawn graph with 15 vertices and 33 edges." />
</div>

This looked pretty good, so I created a cleaner version in <a href="https://app.diagrams.net/">draw.io</a>:

<div align="center">
    <img src="img/20240828_2.png"
        alt="Same graph created with draw.io." />
</div>

Shortly after, I realized that I didn't know how to add a source and a sink to turn the graph into a flow network. After a lot of research, I came across <a href="https://math.stackexchange.com/questions/744276/minimum-cut-algorithm-on-undirected-graph-with-no-source-or-sink">my exact question</a> on Stack Exchange. The answer seems to be to pick any vertex as the sink, and then for each other vertex, compute the maxflow with that vertex as the source. Any of the results with the smallest maxflow will lead to a minimum cut.

## 2024-08-29: First case

I'll turn the test graph into a flow network by choosing one vertex as the source and another as the sink. For more than a day, my curiosity has been nagging me with the question of how to find the _first_ path between the source and the sink. I'm going to leave that unexplored for now — if I'm not mistaken, it will follow the same rules as how to find an _augmenting path_ for any in-progress state of the network. I want to understand what the next step is for at least a few in-progress cases:

- The first path snakes through all of the solution edges, so that finding a maxflow will require reversing the flow across one of the solution edges.
- The first path crosses exactly one solution edge; the source and sink are in different components of the modified graph.
- The source and sink are in the same component, are connected by an edge, and the first path simply traverses that edge.

I felt that this last case would be the best starting example, and I chose `qnr` as the source and `frs` as the sink.

Sidebar on a maximum flow's upper limit: In an unweighted graph, the maximum flow is at most the number of edges coming out of the source, and also at most the number of edges going into the sink. In a weighted graph, it's the total capacity of those edges.

`qnr` has four edges coming out, and `frs` has four edges going in. The maximum flow from `qnr` to `frs` is indeed 4, with the following paths as one possibility:

```
qnr - frs
qnr - rzs - rsh - frs
qnr - cmg - lhk - frs
qnr - nvd - pzl - lsr - frs
```

<div align="center">
    <img src="img/20240829_1.png"
        alt="Test graph with source qnr, sink frs, and the four paths indicated in the text." />
</div>

The first case I wanted to consider is where the first three paths have already been added to the flow network, but `qnr - nvd - pzl - lsr - frs` has not:

<div align="center">
    <img src="img/20240829_2.png"
        alt="Test graph with source qnr, sink frs, and the first three of the paths indicated in the text." />
</div>

And the question is, how do I find an augmenting path for this flow network in this state?

## 2024-08-30: Backward edges

A lightbulb finally went on above my head, about three minutes into <a href="https://www.youtube.com/watch?v=Tl90tNtKvxs">this video</a>:

<div align="center">
    <img src="img/20240830_1.png"
        style="max-height:400px"
        alt="Screenshot from YouTube video where augmenting path is highlighted." />
</div>

It was clear to me that DFS or BFS would always identify an augmenting path, if one existed, using the original flow network — forward edges only — or, in other words, using remaining capacity without undoing any previous flows. But I didn't understand how the use of backward edges would work, as this would undo a previous flow. In the graph above, $A{\rightarrow}D$ is a forward edge, and $D{\rightarrow}A$ is a backward edge. The path $S{\rightarrow}C{\rightarrow}D{\rightarrow}A{\rightarrow}B{\rightarrow}T$ removes 4 from $A{\rightarrow}D$, and what I struggled to understand was how removing 4 here, and adding back that same 4 somewhere else, would result in a net increase instead of no net change.

I finally realized that a new path using a backward edge is equivalent to the new path saying to the existing path, "I've got this covered. From my own sources I can send into this vertex up to X of what you sent into this vertex. Take back as much of that X as you can, and try a different route. If any of it can reach the sink, we've found an augmenting path."

I now realized that my first case is perhaps overly simple, because it can be solved without using any backward paths. Still, I drew the residual network for this network after the first three paths. Because a currently unused edge can be used in either direction — this is an undirected graph — the graph and the residual network include a forward edge in each direction, for any currently unused edge. I've represented unused edges with an arrow at both ends, but the picture would be clearer if I found a way to display two separate arrows, one pointing in either direction.

<div align="center">
    <img src="img/20240830_2.png"
        alt="Residual network for test graph after first three paths." />
</div>

The only edge in the residual network coming out of `qnr` goes to `nvd`, so this would be the first edge traversed by any augmenting path. There are four ways to continue from there, and each can produce an augmenting path. The "I've got this" approach makes me more comfortable with why that is so. For example, the augmenting path `qnr - nvd - lhk - cmg - rzs - lsr - frs` can be thought of as saying to `qnr - cmg - lhk - frs`, "I've got `lhk - frs` covered. Change your path to `qnr - cmg - rzs - lsr - frs`. (And I'll be `qnr - nvd - lhk - frs`. And `cmg - lhk` will go back to being unused.)"

## 2024-08-31 Undirected edges

I feel I'm getting closer to deciding on actual data structures, although I haven't yet specified everything I need those data structures to do. One question I don't know the answer to involves how to handle the two directions of an undirected edge.

I'm not sure how to describe this without using weights, so I'll use weights. Let's say edge $A \textendash B$ has capacity 5, and in the current state $A{\rightarrow}B$ has flow 2. I initially expected the residual network to show $A{\rightarrow}B$ with capacity 3 (this is inarguably correct) and $B{\rightarrow}A$ with capacity 2. But if the maxflow requires a positive flow $B{\rightarrow}A$, that can't be accomplished with a single augmenting path if the residual network's capacity for $B{\rightarrow}A$ is only enough to set the true flow back to zero. It makes more sense to me for the residual network's capacity for $B{\rightarrow}A$ to be 7.

Here's a small example of the same concept:

<div align="center">
    <img src="img/20240831_1.png"
        alt="Minimal example with four vertices to demonstrate having to reverse a flow from an existing path; showing the initial undirected graph, first augmenting path, and residual graph." />
</div>

Sidebar on backward edges involving source or sink: I don't think the residual network needs to show any capacity going back into the source or coming out of the sink, which is why I put the two zeros in the residual network graph. This feels intuitive to me, but I also have an explanation for it. The source has infinite capacity, even if the edges do not, so there's nothing to gain by telling the source to try a different route. The source already uses its infinite capacity to try different routes, to try to find augmenting paths. As for the sink, the goal is to flow into the sink. At that point there's nothing to gain by moving backward, because the flow can't do any better than having reached the sink.

Back to the example: If the residual network's capacity for $B{\rightarrow}A$ was limited to the used capacity on $A{\rightarrow}B$ — 1 rather than 2 — it would take two more augmenting paths to arrive at the maximum flow. The first $S{\rightarrow}B{\rightarrow}A{\rightarrow}T$ would add 1. Then I'd need to observe that $A{\textendash}B$ was empty and add both $A{\rightarrow}B$ and $B{\rightarrow}A$ back to the residual network, each with capacity 1. Finally I could traverse $S{\rightarrow}B{\rightarrow}A{\rightarrow}T$ again to add another 1.

So the approach I'm thinking of is to ensure that, for any undirected edge $A{\textendash}B$, in the residual network the sum of the capacities for $A{\rightarrow}B$ and $B{\rightarrow}A$ is constant at twice the capacity of $A{\textendash}B$. If we add flow to $A{\rightarrow}B$, we subtract that amount from the residual capacity of $A{\rightarrow}B$ and add that amount to the residual capacity of $B{\rightarrow}A$.

At this point I wondered how I would handle a case where a solution edge is not full. A little searching gave me the answer that the edges of a minimum cut will always be saturated, that this case doesn't exist. As I thought about it, it dawned on me that if I drew a graph to represent this case, and then found its actual minimum cut, the unsaturated edge that I had in mind would not actually be a solution edge.

## 2024-09-01: Shortest augmenting path

I was finally ready to think through and adapt Sedgewick's implementation of searching for an augmenting path. If I understand correctly, this is the Edmonds–Karp method, which Sedgewick describes as "breadth-first search (BFS) in the residual network".

1. I will need the adjacency list for the residual network.
2. I will need the current graph of the residual network.
3. For each vertex $B$ other than the source that I come across, I will need the vertex $A$ for which $A{\rightarrow}B$ is the first potential edge to $B$ I identify, and I will need the capacity of $A{\rightarrow}B$ in the residual network. I'll call this data structure $D$ for now, and its identifiers will be the $B$ vertex, the vertex the edge goes _to_.
4. In each round, for each vertex $A$; for each vertex $B$ on the residual adjacency list for $A$; if $B$ is not yet in $D$ and the residual capacity of $A{\rightarrow}B$ is greater than zero, I will add $B$ to $D$ and record the edge's $A$ vertex and capacity, and I will include $B$ in the list of vertices to process (in the $A$ role) in the next round.
5. The source will be the only vertex $A$ in the first round.
6. Rounds will continue until the sink is reached or no vertices are identified for the next round.
7. If the process ends without the sink being reached, there is no augmenting path, and the residual network is in its final state.
8. Otherwise, I will use $D$ to retrace the path from the sink to the source, recording all edges and capacities.
9. I will identify the minimum capacity for the augmenting path.
10. I will update the residual network by subtracting the minimum capacity from each edge in the path and adding the minimum capacity to each reversed edge.

I feel like the code I eventually write may be simpler and clearer. In Python steps 3 through 6 above might look like this:

```python
D = {}
current_round = [source]
next_round = set()
while current_round:
    for A in current_round:
        for B in residual_adj[A]:
            if B not in D:
                capacity = residual_edges.get((A, B), 0)
                if capacity > 0:
                    D[B] = (A, capacity)
                    next_round.add(B)
    if sink in D:
        break
    current_round = list(next_round)
    next_round = set()
```

I am a bit uneasy at leaving the original flow network behind and working only with the residual network. The justification for this may be that the starting residual network _is_ the original flow network.

## 2024-09-02 Minimum cut



For a given source and sink, after finding the residual graph of the maximum flow, to find the mincut I took the approach in <a href="https://stackoverflow.com/questions/4482986/how-can-i-find-the-minimum-cut-on-a-graph-using-a-maximum-flow-algorithm/43210670#43210670">this answer by MichalH</a>. Going back a few days, I chose an arbitrary vertex as the sink and, testing each other vertex as the source, tracked the smallest cutset.

I decided the only data structure I needed was an adjacency list that stored the capacity for each edge. In Python I used a nested dictionary, which for the small example looks like this:

```python
{
    'S': {'A': 1, 'B': 2},
    'A': {'S': 1, 'B': 1, 'T': 2},
    'B': {'S': 2, 'A': 1, 'T': 1},
    'T': {'A': 2, 'B': 1},
}
```

I also used a timer function as a decorator, so I could compare with NetworkX.

In [1]:
from copy import deepcopy
from functools import wraps
import time

In [2]:
def timer(func):
    @wraps(func)
    def wrapper(*arg, **kw):
        start = time.time()
        result = func(*arg, **kw)
        duration = time.time() - start
        print('Time for {0}: {1} seconds'
              .format(func.__name__, round(duration, 1)))
        return result
    return wrapper

In [3]:
@timer
def minimum_cut(adjacency_with_capacity):
    cutset = None
    sink = list(adjacency_with_capacity)[-1]
    for source in [v for v in adjacency_with_capacity if v != sink]:
        R = deepcopy(adjacency_with_capacity)
        while True:
            # find augmenting path with Edmonds–Karp
            D = {}
            current_round = [source]
            next_round = set()
            while current_round:
                for A in current_round:
                    for B, capacity in R[A].items():
                        if B not in D and capacity > 0:
                            D[B] = (A, capacity)
                            next_round.add(B)
                if sink in D:
                    break
                current_round = list(next_round)
                next_round = set()
            if sink not in D:
                break  # no augmenting path
            # update residual network
            minimum_capacity = float('inf')
            edges = []
            B = sink
            A, capacity = D[B]
            while True:  # do while A != source
                minimum_capacity = min(minimum_capacity, capacity)
                edges.append((A, B))
                if A == source:
                    break
                next_A, next_capacity = D[A]
                B, A, capacity = A, next_A, next_capacity
            for A, B in edges:
                R[A][B] -= minimum_capacity
                R[B][A] += minimum_capacity
        # find reachable vertices with BFS
        visited = set()
        visited.add(source)
        current_vertices = [source]
        next_vertices = set()
        while current_vertices:
            for A in current_vertices:
                for B, capacity in R[A].items():
                    if B not in visited and capacity > 0:
                        visited.add(B)
                        next_vertices.add(B)
            current_vertices = list(next_vertices)
            next_vertices = set()
        # find current cutset
        current_cutset = [(A, B)
                          for A in visited
                          for B in adjacency_with_capacity[A]
                          if A in visited
                          and B not in visited]
        if cutset is None or len(current_cutset) < len(cutset):
            cutset = current_cutset
    return cutset

I ran it on the Day 25 test case. After fixing a few bugs, it worked:

In [4]:
from collections import defaultdict

In [5]:
def get_awc_from_aoc_input(s):
    awc = defaultdict(dict)
    for line in s.strip().split('\n'):
        tokens = line.replace(':', '').split()
        A = tokens[0]
        for B in tokens[1:]:
            awc[A][B] = 1
            awc[B][A] = 1
    return awc

In [6]:
test_input = """
jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr
"""

In [7]:
test_awc = get_awc_from_aoc_input(test_input)

In [8]:
minimum_cut(test_awc)

Time for minimum_cut: 0.0 seconds


[('hfx', 'pzl'), ('jqt', 'nvd'), ('bvb', 'cmg')]

Now it was time to try the full input:

In [9]:
with open('../../AdventOfCode/2023/adventofcode.com_2023_day_25_input.txt') as f:
    real_input = f.read()

In [10]:
real_awc = get_awc_from_aoc_input(real_input)

In [11]:
real_mincut = minimum_cut(real_awc)

Time for minimum_cut: 18.1 seconds


And compare with NetworkX:

In [12]:
import networkx as nx
from networkx.algorithms.connectivity import minimum_edge_cut

In [13]:
@timer
def mincut_networkx(s):
    G = nx.Graph()
    for line in s.strip().split('\n'):
        tokens = line.replace(':', '').split()
        A = tokens[0]
        for B in tokens[1:]:
            G.add_edge(A, B, capacity=1)
    return minimum_edge_cut(G)

In [14]:
real_mincut_nx = mincut_networkx(real_input)

Time for mincut_networkx: 6.5 seconds


The solutions matched:

In [15]:
assert sorted(sorted(edge) for edge in real_mincut) \
    == sorted(sorted(edge) for edge in real_mincut_nx)

I was really happy with this. In not much more than 50 lines of code, using simple data structures, I implemented an algorithm that I had struggled on and off for months to understand. Without any optimization, on this decently-sized graph, it took only about three times as long to run as NetworkX.

Could I make my `minimum_cut` function any more concise? Maybe a little, although I'm not playing code golf here. I already used list comprehensions and `defaultdict` for brevity's sake. I could save a line with `visited = set([source])`, but that is a Python-specific workaround for set creation, and reading it confuses me. I also don't think I could cut out much code by assuming an unweighted graph, as I would still need a way to track which edges were full. Tracking capacity achieves this without much overhead.

Could I rewrite this completely from scratch, without looking up anything? Maybe after a couple of attempts. I now understand how the algorithm is built from pieces I was already comfortable with.

I ran one more test with a truly weighted graph. Sedgewick provided <a href="https://algs4.cs.princeton.edu/43mst/mediumEWG.txt">data</a> for a weighted graph with 250 vertices:

In [16]:
from decimal import Decimal  # no floating-point errors for me today, thanks

In [17]:
with open('mediumEWG.txt') as f:
    weighted_input = f.read()

In [18]:
weighted_awc = defaultdict(dict)
for line in weighted_input.strip().split('\n')[2:]:
    A, B, weight = line.split()
    weighted_awc[A][B] = Decimal(weight)
    weighted_awc[B][A] = Decimal(weight)

In [19]:
weighted_mincut = minimum_cut(weighted_awc)
weighted_mincut

Time for minimum_cut: 3.5 seconds


[('25', '111'), ('60', '111')]

There's no reason to expect a unique minimum cut for this graph. Looking back at my implementation, it would be straightforward to accumulate all minimum cuts of smallest size:

In [20]:
@timer
def minimum_cuts(adjacency_with_capacity):
    cutsets = set()                                # changed to set
    size = None                                    # tracking best size
    sink = list(adjacency_with_capacity)[-1]
    for source in [v for v in adjacency_with_capacity if v != sink]:
        R = deepcopy(adjacency_with_capacity)
        while True:
            # find augmenting path with Edmonds–Karp
            D = {}
            current_round = [source]
            next_round = set()
            while current_round:
                for A in current_round:
                    for B, capacity in R[A].items():
                        if B not in D and capacity > 0:
                            D[B] = (A, capacity)
                            next_round.add(B)
                if sink in D:
                    break
                current_round = list(next_round)
                next_round = set()
            if sink not in D:
                break  # no augmenting path
            # update residual network
            minimum_capacity = float('inf')
            edges = []
            B = sink
            A, capacity = D[B]
            while True:  # do while A != source
                minimum_capacity = min(minimum_capacity, capacity)
                edges.append((A, B))
                if A == source:
                    break
                next_A, next_capacity = D[A]
                B, A, capacity = A, next_A, next_capacity
            for A, B in edges:
                R[A][B] -= minimum_capacity
                R[B][A] += minimum_capacity
        # find reachable vertices with BFS
        visited = set()
        visited.add(source)
        current_vertices = [source]
        next_vertices = set()
        while current_vertices:
            for A in current_vertices:
                for B, capacity in R[A].items():
                    if B not in visited and capacity > 0:
                        visited.add(B)
                        next_vertices.add(B)
            current_vertices = list(next_vertices)
            next_vertices = set()
        # find current cutset                      # changed to tuple which is hashable
        current_cutset = tuple(sorted(tuple(sorted((A, B)))
                                      for A in visited
                                      for B in adjacency_with_capacity[A]
                                      if A in visited
                                      and B not in visited))
        if not cutsets:                            # changed to accumulate best cutsets
            cutsets.add(current_cutset)
            size = len(current_cutset)
        elif len(current_cutset) == size:
            cutsets.add(current_cutset)
        elif len(current_cutset) < size:
            cutsets = set([current_cutset])
            size = len(current_cutset)
    return cutsets

In [21]:
minimum_cuts(weighted_awc)

Time for minimum_cuts: 3.5 seconds


{(('111', '25'), ('111', '60')),
 (('111', '61'), ('111', '87')),
 (('127', '20'), ('127', '89'))}

So that's pretty cool — because I wrote it from scratch, it was straightforward to tweak it to answer this related question.

I spent several minutes researching whether NetworkX can do this, and while there is an `all_node_cuts` function, I didn't find an off-the-shelf function to return all minimum edge cuts. To be sure, NetworkX returns one of the solutions in less than one-tenth the time mine did.

In [22]:
weighted_graph = nx.Graph()
for line in weighted_input.strip().split('\n')[2:]:
    A, B, capacity = line.split()
    weighted_graph.add_edge(A, B, capacity=Decimal(capacity))
    weighted_graph.add_edge(B, A, capacity=Decimal(capacity))

In [23]:
%%time
minimum_edge_cut(weighted_graph)

CPU times: user 210 ms, sys: 3.54 ms, total: 214 ms
Wall time: 214 ms


{('111', '25'), ('111', '60')}